# Our database with companies that left Russia in 2022

### База Тиграна с Action, Tone + База старая с INT  + Spark

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from kmodes.kprototypes import KPrototypes

In [2]:
df_int = pd.read_csv("database new.csv")
df_actions_tone = pd.read_csv("new database with Actions.csv")
spark = pd.read_csv("spark.csv", delimiter=';')

FileNotFoundError: [Errno 2] No such file or directory: 'database new.csv'

In [ ]:
df_int.shape

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", df_int.isnull().sum())

In [ ]:
df_actions_tone.shape

In [ ]:
df_actions_tone = df_actions_tone.drop('Unnamed: 0', axis = 1)
df_actions_tone = df_actions_tone.rename(columns = {'tone': 'Tone'})
df_actions_tone = df_actions_tone.sort_values(by='Company')
df_actions_tone = df_actions_tone.reset_index(drop=True)
df_actions_tone.head(1)

In [ ]:
# Define a dictionary to map the numbers to their corresponding labels
label_mapping = {0: 'leaved', 1: 'continue', 2: 'paused'}

# Replace the numbers in the specified column with the labels
df_actions_tone['Tone'] = df_actions_tone['Tone'].map(label_mapping)

In [ ]:
df_actions_tone.head(10)

In [ ]:
df_int = df_int.rename(columns={'i': 'Company'})
df_int = df_int.rename(columns={'ИНН': 'INT'})
df_int = df_int.sort_values(by='Company')
df_int = df_int.reset_index(drop=True)
df_int.head(1)

In [ ]:
df1 = df_actions_tone.copy()
df2 = df_int.copy()

# Merge the two datasets based on the "Company" column
data = pd.merge(df1, df2, on='Company')
data.head(1)

In [ ]:
# Select the desired columns
desired_columns = ['Company', 'Action_x', 'Industry_x', 'Country_x', 'Tone', 'INT']
data = data[desired_columns]
data.head(1)

In [ ]:
# Rename the columns
data.columns = ['Company', 'Action', 'Industry', 'Country', 'Tone', 'INT']

In [ ]:
data

# Spark database

In [ ]:
spark.head()

In [ ]:
spark.shape

<span style="font-size: 17px;"> Delete all unnecessary columns

In [ ]:
columns_to_drop = ['№', 'Наименование', 'Регистрационный номер', 'Краткое наименование', 'Дата регистрации']
spark = spark.drop(columns_to_drop, axis=1)

In [ ]:
print(spark.columns)

<span style="font-size: 17px;"> Rename columns 'Наименование на английском' and 'Код налогоплательщика' to 'Company' and 'INT'

In [ ]:
spark = spark.rename(columns={"Наименование на английском": "Company"})
spark = spark.rename(columns={"Код налогоплательщика": "INT"})
# spark['Company'] = spark['Company'].str.lower()

In [ ]:
spark.head(1)

# Merging two databases into one

In [ ]:
df_merged = data.merge(spark, on='INT')

In [ ]:
df_merged.head()

In [ ]:
df_merged = df_merged.rename(columns={"Company_y": "Company"})

df_merged = df_merged.drop({'Company_x', 'INT', 
                          '2017, Среднесписочная численность работников', 
                          '2018, Среднесписочная численность работников', 
                          '2019, Среднесписочная численность работников',
                          '2020, Среднесписочная численность работников', 
                          '2021, Среднесписочная численность работников',
                          'Мои списки'}, axis=1)

In [ ]:
df_merged = df_merged[['Company', 'Industry', 'Country', 'Action', 'Tone', 'Возраст компании, лет',
       'Вид деятельности/отрасль', 'Организационно-правовая форма',
       'Форма собственности', 'Размер компании', '2017, Выручка, RUB',
       '2018, Выручка, RUB', '2019, Выручка, RUB', '2020, Выручка, RUB',
       '2021, Выручка, RUB', '2017, Прибыль (убыток) до налогообложения , RUB',
       '2018, Прибыль (убыток) до налогообложения , RUB',
       '2019, Прибыль (убыток) до налогообложения , RUB',
       '2020, Прибыль (убыток) до налогообложения , RUB',
       '2021, Прибыль (убыток) до налогообложения , RUB']]

In [ ]:
df_merged.head()

In [ ]:
database = df_merged.copy()

<span style="font-size: 17px;"> The column 'Возраст компании, лет' is of string datatype. To make it float datatype, we replace , with . 

In [ ]:
if database['Возраст компании, лет'].dtype == object:
        # Replace commas with periods in the column
        database['Возраст компании, лет'] = database['Возраст компании, лет'].str.replace(',', '.')
database['Возраст компании, лет'] = database['Возраст компании, лет'].astype(float)

<span style="font-size: 17px;"> Some columns has big number with spaces, therefore they are of a string type.  To make them float datatype, we delete all unnecassary spaces

In [ ]:
database['2017, Выручка, RUB'] = database['2017, Выручка, RUB'].str.replace(' ', '').astype(float)
database['2018, Выручка, RUB'] = database['2018, Выручка, RUB'].str.replace(' ', '').astype(float)
database['2019, Выручка, RUB'] = database['2019, Выручка, RUB'].str.replace(' ', '').astype(float)
database['2020, Выручка, RUB'] = database['2020, Выручка, RUB'].str.replace(' ', '').astype(float)
database['2021, Выручка, RUB'] = database['2021, Выручка, RUB'].str.replace(' ', '').astype(float)
database['2017, Прибыль (убыток) до налогообложения , RUB'] = database['2017, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
database['2018, Прибыль (убыток) до налогообложения , RUB'] = database['2018, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
database['2019, Прибыль (убыток) до налогообложения , RUB'] = database['2019, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
database['2020, Прибыль (убыток) до налогообложения , RUB'] = database['2020, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)
database['2021, Прибыль (убыток) до налогообложения , RUB'] = database['2021, Прибыль (убыток) до налогообложения , RUB'].str.replace(' ', '').astype(float)

In [ ]:
database.head(5)

In [ ]:
database.shape

# Handling missing values

# Сделать прибыль и выручку из средних за все года

<span style="font-size: 17px;"> Count total NaN at each column in a database

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", database.isnull().sum())

<span style="font-size: 17px;"> Visualize missing values in dataset

In [ ]:
pip install missingno

<span style="font-size: 17px;"> Visualize the number of missing values as a bar chart

In [ ]:
import missingno as msno
msno.bar(database)

<span style="font-size: 17px;"> Visualize the correlation between the number of missing values in different columns as a heatmap.

<span style="font-size: 17px;"> As we see below, there is a diagonal with values 0.9, which means that profit and insome highly correlates in terms of null values (if profit value is present, then income value is likely to be present too)

In [ ]:
msno.heatmap(database)

# Here we have to fill NaN values of proft with mean !!!!!

<span style="font-size: 17px;"> Delete all rows where more than 6 NaN values exist

In [ ]:
database.dropna(thresh=database.shape[1] - 6, inplace=True)

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", database.isnull().sum())

In [ ]:
database.shape

<span style="font-size: 17px;"> Delete all rows where Nan values exist

In [ ]:
database['Mean Revenue'] = database[['2017, Выручка, RUB', '2018, Выручка, RUB', '2019, Выручка, RUB', '2020, Выручка, RUB', '2021, Выручка, RUB']].mean(axis=1)

In [ ]:
database.dropna(subset=['Mean Revenue'], inplace=True)
database.drop(['2017, Выручка, RUB', '2018, Выручка, RUB', '2019, Выручка, RUB', '2020, Выручка, RUB', '2021, Выручка, RUB'], axis=1, inplace=True)


In [ ]:
database.head()

In [ ]:
database.shape

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", database.isnull().sum())

In [ ]:
database['Mean Profit'] = database[['2017, Прибыль (убыток) до налогообложения , RUB', '2018, Прибыль (убыток) до налогообложения , RUB', '2019, Прибыль (убыток) до налогообложения , RUB', '2020, Прибыль (убыток) до налогообложения , RUB', '2021, Прибыль (убыток) до налогообложения , RUB']].mean(axis=1)

In [ ]:
database.dropna(subset=['Mean Revenue'], inplace=True)
database.drop(['2017, Прибыль (убыток) до налогообложения , RUB', '2018, Прибыль (убыток) до налогообложения , RUB', '2019, Прибыль (убыток) до налогообложения , RUB', '2020, Прибыль (убыток) до налогообложения , RUB', '2021, Прибыль (убыток) до налогообложения , RUB'], axis=1, inplace=True)

In [ ]:
database.head()

In [ ]:
database.shape

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", database.isnull().sum())

In [ ]:
database.dropna(inplace=True)

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", database.isnull().sum())

In [ ]:
database.shape

In [ ]:
database.describe()

In [ ]:
database.corr()

<span style="font-size: 17px;">  Now our dataset has no NaN values and we can start clustarisation

# Clusterisation (k-prototypes)

Если мы оставляем колонку 'Mean Profit', то получается 1 кластер

Если мы удаляем колонку 'Mean Revenue' and 'Mean Profit', то получается 2 кластера: 
0 : 326 и 1 : 218

In [ ]:
database2 = database.copy()
database2 = database2.drop({'Company', 'Action', 'Вид деятельности/отрасль','Организационно-правовая форма'} , axis = 1)
# database2 = database2.drop({'Company', 'Action','Вид деятельности/отрасль', 'Организационно-правовая форма','Форма собственности'} , axis = 1)

In [ ]:
database2.head(5)

In [ ]:
pip install kmodes

In [ ]:
mark_array=database2.values

<span style="font-size: 17px;"> Mark columns that has float datatypes

In [ ]:
mark_array[:, 3] = mark_array[:, 3].astype(float)
# mark_array[:, 5] = mark_array[:, 5].astype(float)
# mark_array[:, 6] = mark_array[:, 6].astype(float)
# mark_array[:, 8] = mark_array[:, 8].astype(float)
# mark_array[:, 9] = mark_array[:, 9].astype(float)

In [ ]:
mark_array

<span style="font-size: 17px;"> Get the position of categorical columns

In [ ]:
catColumnsPos = [database2.columns.get_loc(col) for col in list(database2.select_dtypes('object').columns)]

print('Categorical columns           : {}'.format(list(database2.select_dtypes('object').columns)))
print('Categorical columns position  : {}'.format(catColumnsPos))

In [ ]:
# Convert dataframe to matrix
dfMatrix = database2.to_numpy()

In [ ]:
dfMatrix 

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", database2.isnull().sum())

<span style="font-size: 17px;"> Elbow method to detect number of clusters K

In [ ]:
from kmodes.kprototypes import KPrototypes

cost = []
for cluster in range(1, 10):
    try:
        kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
        kprototype.fit_predict(database2, categorical = catColumnsPos)
        cost.append(kprototype.cost_)
        print('Cluster initiation: {}'.format(cluster))
    except:
        break

plt.plot(cost)
plt.xlabel('K')
plt.ylabel('cost')
plt.show

In [ ]:
cost

In [ ]:
kproto = KPrototypes(n_clusters=2, verbose=2,max_iter=30)
clusters = kproto.fit_predict(mark_array, categorical=[0, 1, 2, 4, 5, 6])

In [ ]:
print(kproto.cluster_centroids_)

In [ ]:
cluster_dict=[]
for c in clusters:
    cluster_dict.append(c)

In [ ]:
cluster_dict

In [ ]:
database2['cluster']=cluster_dict

In [ ]:
for i in range(0,5):
    print(i, ": ", database2[database2['cluster']== i].value_counts().sum())

In [ ]:
database2

In [ ]:
database2[database2['cluster']== 0].head(10)

In [ ]:
database2[database2['cluster']== 1].head(10)

In [ ]:
database2[database2['cluster']== 2].head(10)

<span style="font-size: 17px;"> Visualisation:

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

database2.columns = ["Action", "Tone", "Возраст компании, лет", "Вид деятельности/отрасль", "Организационно-правовая форма", "Форма собственности", "cluster"]
sns.pairplot(database2, hue="cluster")

In [ ]:
sns.boxplot(x="cluster", y="Возраст компании, лет", data=database2)


In [ ]:
sns.boxplot(x="cluster", y="Организационно-правовая форма", data=database2)

In [ ]:
sns.boxplot(x="cluster", y="Форма собственности", data=database2)

## Clusterisation of dataset with income and profit

In [ ]:
df_filtered = database.copy()
df_filtered = df_filtered.drop({'Company', 'Action', 'Вид деятельности/отрасль'}, axis=1)

In [ ]:
df_filtered.head(1)

In [ ]:
df_cat = df_filtered[['Industry', 'Country', 'Tone', 'Организационно-правовая форма', 'Форма собственности', 'Размер компании']]

for i in df_cat.columns:
    print(df_cat[i].unique())

In [ ]:
pip install kmodes

<span style="font-size: 17px;"> Check that there are no NaN values in the dataset</span>

In [ ]:
print(" \nCount total NaN at each column in a DataFrame : \n\n", df_filtered.isnull().sum())

In [ ]:
mark_array=df_filtered.values

<span style="font-size: 17px;"> Mark columns that has float datatypes

In [ ]:
mark_array[:, 2] = mark_array[:, 2].astype(float)
mark_array[:, 7] = mark_array[:, 7].astype(float)
mark_array[:, 8] = mark_array[:, 8].astype(float)
mark_array[:, 9] = mark_array[:, 9].astype(float)
mark_array[:, 10] = mark_array[:, 10].astype(float)
mark_array[:,11] = mark_array[:, 11].astype(float)

<span style="font-size: 17px;"> Get the position of categorical columns

In [ ]:
catColumnsPos = [df_filtered.columns.get_loc(col) for col in list(df_filtered.select_dtypes('object').columns)]

print('Categorical columns           : {}'.format(list(df_filtered.select_dtypes('object').columns)))
print('Categorical columns position  : {}'.format(catColumnsPos))

In [ ]:
# Convert dataframe to matrix
dfMatrix = df_filtered.to_numpy()

<span style="font-size: 17px;"> Elbow method to detect number of clusters K

In [ ]:
from kmodes.kprototypes import KPrototypes

cost = []
for cluster in range(1, 10):
    try:
        kprototype = KPrototypes(n_jobs = -1, n_clusters = cluster, init = 'Huang', random_state = 0)
        kprototype.fit_predict(dfMatrix, categorical = catColumnsPos)
        cost.append(kprototype.cost_)
        print('Cluster initiation: {}'.format(cluster))
    except:
        break

plt.plot(cost)
plt.xlabel('K')
plt.ylabel('cost')
plt.show

<span style="font-size: 17px;">  The elbow is at point 3 => numbers of clusters = 3

In [ ]:
mark_array

In [ ]:
kproto = KPrototypes(n_clusters=3, verbose=2,max_iter=30)
clusters = kproto.fit_predict(mark_array, categorical=[0, 1, 4, 5, 6])

In [ ]:
print(kproto.cluster_centroids_)

In [ ]:
cluster_dict=[]
for c in clusters:
    cluster_dict.append(c)

In [ ]:
cluster_dict

In [ ]:
df_filtered['Cluster']=cluster_dict

In [ ]:
df_filtered

In [ ]:
df_filtered[df_filtered['Cluster']== 0].head(10)

In [ ]:
df_filtered[df_filtered['Cluster']== 1].head(10)

In [ ]:
df_filtered[df_filtered['Cluster']== 2].head(10)

<span style="font-size: 17px;"> Let's visualize our clusters

In [ ]:
#the volume of each cluster
df_filtered['Cluster'].value_counts().plot(kind='bar')

plt.xticks(fontsize=10, rotation=0, ha='right')

# Graphs

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
database3 = database.copy()
database3

### Countries

<span style="font-size: 17px;"> Value_counts() returns the count of unique values in the 'Country' column, and head(10) selects the first 10 values, which represent the top 10 countries with the highest counts.</span>

In [ ]:
df_countries = database3['Country'].value_counts().head(10)

In [ ]:
df_countries = df_countries.to_frame().reset_index()

In [ ]:
df_countries

In [ ]:
# Increase the figure size and adjust the spacing
plt.figure(figsize=(12, 8))
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

# Plot the bar chart
df_countries.plot(kind='bar', x='index', y='Country', color='pink')

# Set the labels and title
plt.xlabel('Country', fontsize=12, labelpad=10)
plt.ylabel('Number of Companies', fontsize=12, labelpad=10)
plt.title('Number of Companies per Country', fontsize=14, fontweight='bold', pad=20)

# Adjust the font size and rotation of the x-axis labels
plt.xticks(fontsize=10, rotation=45, ha='right')

# Adjust the font size of the y-axis labels
plt.yticks(fontsize=10)

# Remove the top and right spines (borders)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add grid lines to the plot
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()

### Industries:

In [ ]:
df_industries = database3['Industry'].value_counts().head(15)
df_industries = df_industries.to_frame().reset_index()

In [ ]:
import matplotlib.pyplot as plt

# Increase the figure size and adjust the spacing
plt.figure(figsize=(16, 8))
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

# Plot the bar chart
df_industries.plot(kind='bar', x='index', y='Industry', color='#a7e8a9')

# Set the labels and title
plt.xlabel('Industry', fontsize=12, labelpad=10)
plt.ylabel('Number of Companies', fontsize=12, labelpad=10)
plt.title('Number of Companies per Industry', fontsize=14, fontweight='bold', pad=20)

# Adjust the font size and rotation of the x-axis labels
plt.xticks(fontsize=10, rotation=45, ha='right')

# Adjust the font size of the y-axis labels
plt.yticks(fontsize=10)

# Remove the top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add grid lines to the plot
plt.grid(axis='y', linestyle='--', alpha=0.5)

plt.show()

### Size

In [ ]:
df_size = database3['Размер компании'].value_counts()
df_size = df_size.to_frame().reset_index()

In [ ]:
import matplotlib.pyplot as plt

# Increase the figure size and adjust the spacing
plt.figure(figsize=(14, 8))
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

# Plot the bar chart
df_size.plot(kind='bar', x='index', y='Размер компании', color='#b6dffc')

# Set the labels and title
plt.xlabel('Size', fontsize=12, labelpad=10)
plt.ylabel('Number of Companies', fontsize=12, labelpad=10)
plt.title('Number of Companies per Size', fontsize=14, fontweight='bold', pad=20)

# Adjust the font size and rotation of the x-axis labels
plt.xticks(fontsize=10, rotation=45, ha='right')

# Adjust the font size of the y-axis labels
plt.yticks(fontsize=10)

# Remove the top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add grid lines to the plot
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()

In [ ]:
database3.head()

### Age

In [ ]:
database3['Возраст компании, лет'].plot(kind='hist', bins=30)

plt.xlabel('Age of Company')
plt.ylabel('Count')
plt.title('Distribution of Company Age')

plt.show()

### Organisational form

In [ ]:
df_form = database3['Организационно-правовая форма'].value_counts().head(5)
df_form = df_form.to_frame().reset_index()
df_form

In [ ]:
# Increase the figure size and adjust the spacing
plt.figure(figsize=(10, 5))
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

# Plot the bar chart
df_form.plot(kind='bar', x='index', y='Организационно-правовая форма', color='#ccccff')

# Set the labels and title
plt.xlabel('Type of form', fontsize=12, labelpad=10)
plt.ylabel('Number of Companies', fontsize=12, labelpad=10)
plt.title('Number of Companies per Organisational form', fontsize=14, fontweight='bold', pad=20)

# Adjust the font size and rotation of the x-axis labels
plt.xticks(fontsize=10, rotation=45, ha='right')

# Adjust the font size of the y-axis labels
plt.yticks(fontsize=10)

# Remove the top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add grid lines to the plot
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()

### Form of property

In [ ]:
df_form2 = database3['Форма собственности'].value_counts().head(5)
df_form2 = df_form2.to_frame().reset_index()
df_form2

In [ ]:
# Increase the figure size and adjust the spacing
plt.figure(figsize=(10, 5))
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

# Plot the bar chart
df_form2.plot(kind='bar', x='index', y='Форма собственности', color='#ffcc99')

# Set the labels and title
plt.xlabel('Type of form', fontsize=12, labelpad=10)
plt.ylabel('Number of Companies', fontsize=12, labelpad=10)
plt.title('Number of Companies per Form of property', fontsize=14, fontweight='bold', pad=20)

# Adjust the font size and rotation of the x-axis labels
plt.xticks(fontsize=10, rotation=45, ha='right')

# Adjust the font size of the y-axis labels
plt.yticks(fontsize=10)

# Remove the top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add grid lines to the plot
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()

### Tone
* 0 - completely stoped operations in Russia
* 1 - continue operations in Russia
* 2 - paused operations in Russia

In [ ]:
df_tone = database3['Tone'].value_counts().head(5)
df_tone

In [ ]:
# Increase the figure size and adjust the spacing
plt.figure(figsize=(10, 5))
plt.subplots_adjust(left=0.1, right=0.9, top=0.9, bottom=0.1)

# Plot the bar chart
df_tone.plot(kind='bar', x='index', y='Tone', color='#ffcc99')

# Set the labels and title
plt.xlabel('Tone', fontsize=12, labelpad=10)
plt.ylabel('Number of Companies', fontsize=12, labelpad=10)
plt.title('Number of Companies per Tone', fontsize=14, fontweight='bold', pad=20)

# Adjust the font size and rotation of the x-axis labels
plt.xticks(fontsize=10, rotation=0, ha='right')

# Adjust the font size of the y-axis labels
plt.yticks(fontsize=10)

# Remove the top and right spines
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

# Add grid lines to the plot
plt.grid(axis='y', linestyle='--', alpha=0.5)

# Show the plot
plt.show()

# Prediction of Tone

In [3]:
df = database.copy()

NameError: name 'database' is not defined

In [ ]:
df.head()

In [ ]:
df.columns

### Before predicting, we should check whether variables "Mean Revenue", "Country" and "Industry" are somehow associated with "Tone"

In [ ]:
from scipy.stats import chi2_contingency

### "Tone" and "Mean Revenue"

In [ ]:
# Create a contingency table between "Tone" and "Mean Revenue"
contingency_table = pd.crosstab(df['Tone'], df['Mean Revenue'])

# Perform the chi-square test of independence
chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)

The p-value, 0.08029718074555114, represents the statistical significance of the association. In this case, the p-value is above the typical significance level of 0.05, suggesting that there is not enough evidence to reject the null hypothesis of independence. This means that based on the available data, <span style="font-size: 17px;">  we do not have significant evidence to conclude that there is an association between the "Tone" and "Mean Revenue" variables.

### "Tone" and "Country"

In [ ]:
# Create a contingency table between "Tone" and "Country"
contingency_table = pd.crosstab(df['Tone'], df['Country'])

# Perform the chi-square test of independence
chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)

The p-value is extremely small (approximately 1.0227188469211595e-09), which is significantly below the typical significance level of 0.05. This indicates strong evidence to reject the null hypothesis of independence. Therefore, based on the available data, we can conclude that <span style="font-size: 17px;"> there is a significant association between the "Tone" and "Country" variables.

### "Tone" and "Industry"

In [ ]:
# Create a contingency table between "Tone" and "Industry"
contingency_table = pd.crosstab(df['Tone'], df['Industry'])

# Perform the chi-square test of independence
chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)

The p-value is extremely small (approximately 4.815758757819452e-09), which is significantly below the typical significance level of 0.05. This provides strong evidence to reject the null hypothesis of independence. Therefore, based on the available data, we can conclude that <span style="font-size: 17px;">  there is a significant association between the "Tone" and "Industry" variables.

### "Tone" and "Age"

In [ ]:
# Create a contingency table between "Tone" and "Industry"
contingency_table = pd.crosstab(df['Tone'], df['Возраст компании, лет'])

# Perform the chi-square test of independence
chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)

<span style="font-size: 17px;"> no dependence

### "Tone" and "Форма собственности"

In [ ]:
# Create a contingency table between "Tone" and "Industry"
contingency_table = pd.crosstab(df['Tone'], df['Форма собственности'])

# Perform the chi-square test of independence
chi2, p_value, _, _ = chi2_contingency(contingency_table)

print("Chi-square statistic:", chi2)
print("p-value:", p_value)

<span style="font-size: 17px;"> no dependence

### Therefore, we can predict Tone only by Country and by Industry

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [ ]:
# Split the dataset into input features and target variable
X = df[['Country', 'Industry']]  # Input features
y = df['Tone']                   # Target variable

In [ ]:
# Choose a machine learning algorithm and define the preprocessing steps
model = DecisionTreeClassifier()

In [ ]:
# One-hot encoding for categorical variables
categorical_features = ['Country', 'Industry']
preprocessor = ColumnTransformer(
    transformers=[('cat', OneHotEncoder(), categorical_features)])

In [ ]:
# Combine preprocessing and model into a single pipeline
model = Pipeline(steps=[('preprocessor', preprocessor),
                           ('model', model)])

In [ ]:
# Train the model
model.fit(X, y)

In [ ]:
# Predict the tone for new companies
new_companies = pd.DataFrame({'Country': ['Canada', 'France'],
                              'Industry': ['Industrials', 'IT']})
predictions = model.predict(new_companies)

# Print the predicted tones for the new companies
print(predictions)